<a href="https://colab.research.google.com/github/shrutin567/Earth-Observation/blob/main/download_Sentinel_NDVI_Image_Collection_batch_Export_in_GEE_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import ee engine
import ee

#trigger the authentification flow
ee.Authenticate()

#initializae the library
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=AuB9Jsqj7MNGCZpjwOg3qnb70ZR_f8JTNM4RWCmftUg&tc=OaNla6dGErhnJEh7tlFO6mBY5Bq5rbB0OT1p497tdgw&cc=AKqjh5mfFk8RcYuUaVlJjtVfn1W6uHmxMBVcREwQFjE

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1Adeu5BVN44VrxUw-1U2i1AGE2r9SEzctWQNwU1-ztRvXyFk2hvQljjr0Ibo

Successfully saved authorization token.


In [ ]:
geometry=ee.Geometry.Point([31.5051, -26.6061])
#location = roi.geometry().centroid().coordinates().getInfo()[::-1]
location=geometry.coordinates().getInfo()[::-1]

s2=ee.ImageCollection('COPERNICUS/S2_HARMONIZED')
rgbVis={
    'min':0.0,
    'max':3000,
    'bands':['B4','B3','B2'],
}
#write a function for cloud masking
def maskS2clouds(image):
  qa=image.select('QA60')
  cloudBitMask=1<<10
  cirrusBitMask=1<<11
  mask=qa.bitwiseAnd(cloudBitMask).eq(0).And(
      qa.bitwiseAnd(cirrusBitMask).eq(0))
  return image.updateMask(mask) \
  .select("B.*") \
  .copyProperties(image,["system:time_start"])

filtered=s2.filter(ee.Filter.date('2020-01-01','2020-05-01')) \
.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',30)) \
.filter(ee.Filter.bounds(geometry)) \
.map(maskS2clouds)

#write a function that compares NDVI for an image and adds it as a band
def addNDVI(image):
  ndvi=image.normalizedDifference(['B5','B4']).rename('ndvi')
  return image.addBands(ndvi)

withNdvi=filtered.map(addNDVI)
median=withNdvi.median()

In [ ]:
#import the folium library
import folium

#define a method for displaying ee image tiles to folium map
def addLayer(self,ee_image_object,vis_params,name):
  map_id_dict=ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href= "https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

#add ee drawing method to folium
folium.Map.addLayer=addLayer

#create the map object
Map=folium.Map(location=location,zoom_start=10)

In [ ]:
Map.addLayer(median,rgbVis,'Sentinel')
#add a layer control panel to the map
Map.add_child(folium.LayerControl())

#display the map
display(Map)

In [ ]:
image_ids=withNdvi.aggregate_array('system:index').getInfo()
print('Total images: ',len(image_ids))

Total images:  11


In [ ]:
#export with 100m resolution
for i, image_id in enumerate(image_ids):
  image=ee.Image(withNdvi.filter(ee.Filter.eq('system:index',image_id)).first())
  task=ee.batch.Export.image.toDrive(**{
      'image':image.select('ndvi'),
      'description':image.id().getInfo(),
      'folder':'Example_folder',
      'scale':100,
      'region':image.geometry().bounds().getInfo()['coordinates'],
      'maxPixels':1e10
  })
  task.start()
  print('Started Task: ',i+1)

Started Task:  1
Started Task:  2
Started Task:  3
Started Task:  4
Started Task:  5
Started Task:  6
Started Task:  7
Started Task:  8
Started Task:  9
Started Task:  10
Started Task:  11
